Classic Neural Network - Sanity Check

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Generate random data
def generate_data(size):
    x = np.random.randn(size, 2)
    y = np.tanh(x[:, 0] + x[:, 1])
    y = 1. / (1. + np.exp(-(y + y)))
    y = y > 0.5
    return x, y

In [3]:
# Define Variables
iterations = 100000
number_of_samples = 100
# Define training data
X_train, Y_train = generate_data(100)

In [4]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))
# Define weights
w11 = np.random.randn(1, 100)
w12 = np.random.randn(1, 100)
w21 = np.random.randn(1, 100)
w22 = np.random.randn(1, 100)
w31 = np.random.randn(1, 100)
w32 = np.random.randn(1, 100)
x1 = X_train[:, 0]
x2 = X_train[:, 1]
# Define the layer
l0 = []

In [5]:
for _ in range(iterations):
    x3 = np.tanh(w11 * x1 + w12 * x2)
    x4 = np.tanh(w21 * x1 + w22 * x2)
    l0 = sigmoid(w31 * x3 + w32 * x4)
    l1_error = Y_train - l0
    l1_delta = l1_error * sigmoid(l0)
    w11 += x1 * l1_delta
    w12 += x2 * l1_delta
    w21 += x1 * l1_delta
    w22 += x2 * l1_delta
    w31 += x1 * l1_delta
    w32 += x2 * l1_delta

In [6]:
# Print the accuracy of train data and test data
print("\nOne sample of weights (w11, w12, w21, w22, w31, w32):\n", w11[-1][-1], w12[-1][-1], w21[-1][-1], w22[-1][-1],
      w31[-1][-1], w32[-1][-1])
l0[l0 < 0.5] = 0
l0[l0 >= 0.5] = 1
print("Accuracy on train data: ", (Y_train == l0).mean())

X_test, Y_test = generate_data(100)
x1 = X_test[:, 0]
x2 = X_test[:, 1]
x3 = np.tanh(w11 * x1 + w12 * x2)
x4 = np.tanh(w21 * x1 + w22 * x2)
l1_test = sigmoid(w31 * x3 + w32 * x4)
l1_test[l1_test < 0.5] = 0
l1_test[l1_test >= 0.5] = 1
print("Accuracy on test data: ", (Y_test == l1_test).mean())


One sample of weights (w11, w12, w21, w22, w31, w32):
 10.701910685577838 1.3431029919992339 10.771154790127595 -0.9064395202316625 10.143985352954644 2.083379507181506
Accuracy on train data:  1.0
Accuracy on test data:  0.76


Classic Neural Network - Binary classification

In [7]:
import scipy
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [8]:
def sigmoid(x, deriv=False):
    if deriv:
        return x * (1 - x)
    return scipy.special.expit(x)

In [9]:
# Define variables
n_samples = 1200
iterations = 10000
number_of_samples = np.int32(n_samples / 2)
number_of_features = 2

In [10]:
# Prepare the training and testing data 
X_data, Y_data = make_moons(noise=0.2, random_state=0, n_samples=n_samples)
X_data = scale(X_data)
X_data = X_data.astype(np.double)
Y_data = Y_data.astype(np.double)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=.5)
Y_train = np.reshape(Y_train, [number_of_samples, 1])

In [11]:
# Define weights
w_l0 = 2 * np.random.random((number_of_features, number_of_samples)) - 1
w_l1 = 2 * np.random.random((number_of_samples, 1)) - 1
# Define the 3 layers of the network
l0 = []
l1 = []
l2 = []

In [12]:
for index in range(iterations):
    # The first layer is defined by the training data
    l0 = X_train

    # The second and third layer is the result of the sigmoid function with the weights in the first and second layer 
    l1 = sigmoid(np.dot(l0, w_l0))
    l2 = sigmoid(np.dot(l1, w_l1))

    # Calculate the error compared to the training data 
    l2_error = Y_train - l2
    if (index % 1000) == 0:
        print("Loss:" + str(np.mean(np.abs(l2_error))))

    # Calculate network error
    l2_delta = l2_error * sigmoid(l2)
    l1_error = l2_delta.dot(w_l1.T)
    l1_delta = l1_error * sigmoid(l1)

    # Update the weights to minimize the error compared to the training data
    w_l0 += l0.T.dot(l1_delta)
    w_l1 += l1.T.dot(l2_delta)

Loss:0.22347290461399794
Loss:0.13000000000103806
Loss:0.13166666666666665
Loss:0.299998766642782
Loss:0.20156510086872512
Loss:0.16166667274142207
Loss:0.12666666666666668
Loss:0.12833333333333333
Loss:0.135
Loss:0.13


In [13]:
# Check the accuracy 
y_pred = l2
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

l0 = X_test
l1 = sigmoid(np.dot(l0, w_l0))
l2 = sigmoid(np.dot(l1, w_l1))

y_pred_test = l2
y_pred_test[y_pred_test <= 0.5] = 0
y_pred_test[y_pred_test > 0.5] = 1

In [14]:
print("Accuracy on train data: ", (Y_train == y_pred).mean())
print("Accuracy on test data: ", (Y_test == y_pred_test).mean())

Accuracy on train data:  0.87
Accuracy on test data:  0.4918888888888889


Classic Neural Network - Multiclass classification

In [15]:
import scipy
from sklearn.datasets import load_iris

In [16]:
def sigmoid(x, deriv=False):
    if deriv:
        return x * (1 - x)
    return scipy.special.expit(x)

In [17]:
# Define variables
n_samples = 150
iterations = 10000
number_of_samples = np.int32(n_samples / 2)
number_of_features = 2

In [18]:
# Prepare training and testing data
iris = load_iris()
X_data = iris.data[:, :2]
Y_data = iris.target
X_data = scale(X_data)
X_data = X_data.astype(float)
Y_data = Y_data.astype(float)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=.5)
Y_train = np.reshape(Y_train, [number_of_samples, 1])

In [19]:
# Define weights
w_l0 = 2 * np.random.random((number_of_features, number_of_samples)) - 1
w_l1 = 2 * np.random.random((number_of_samples, 1)) - 1
# Define network layers
l0 = []
l1 = []
l2 = []

In [20]:
for index in range(iterations):
  # The first layer is represented by the training data
  l0 = X_train
  # The second and third layer is the result of the sigmoid function with the weights in the first and second layer
  l1 = sigmoid(np.dot(l0, w_l0))
  l2 = sigmoid(np.dot(l1, w_l1))
  
  # The error compared to the training data 
  l2_error = Y_train - l2
  if (index % 1000) == 0:
    print("Loss:" + str(np.mean(np.abs(l2_error))))

  # Network error
  l2_delta = l2_error * sigmoid(l2)
  l1_error = l2_delta.dot(w_l1.T)
  l1_delta = l1_error * sigmoid(l1)
  
  # Update the weights to minimize the error compared to the training data 
  w_l1 += l1.T.dot(l2_delta)
  w_l0 += l0.T.dot(l1_delta)

Loss:0.7981589028478003
Loss:0.39186999647699194
Loss:0.39186999647699194
Loss:0.39186999647699194
Loss:0.39186999647699194
Loss:0.39186999647699194
Loss:0.39186999555007135
Loss:0.3490598303082677
Loss:0.3490598290598291
Loss:0.3490598290598291


In [21]:
# Check accuracy
y_pred_train = l2
y_pred_train[y_pred_train <= 3] = 2
y_pred_train[y_pred_train <= 2] = 1
y_pred_train[y_pred_train <= 1] = 0

l0 = X_test
l1 = sigmoid(np.dot(l0, w_l0))
l2 = sigmoid(np.dot(l1, w_l1))

y_pred_test = l2
y_pred_test[y_pred_test <= 3] = 2
y_pred_test[y_pred_test <= 2] = 1
y_pred_test[y_pred_test <= 1] = 0

print("Accuracy on train data: ", (Y_train == y_pred_train).mean())
print("Accuracy on test data: ", (Y_test == y_pred_test).mean())

Accuracy on train data:  0.4666666666666667
Accuracy on test data:  0.2
